In [17]:
#!/usr/bin/env python3
import os
import shutil
import pandas as pd
import math
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment
from openpyxl.utils import get_column_letter

# === Einstellungen ===
FONT = Font(name='Arial', size=8)
FONT_BOLD = Font(name='Arial', size=8, bold=True)
thin = Side(border_style='thin', color='000000')
border = Border(left=thin, right=thin, top=thin, bottom=thin)

# Hilfsfunktion: Formatiert Zahl mit sig signifikanten Ziffern, Komma als Dezimaltrennzeichen
def format_sig(x, sig=4):
    if pd.isna(x):
        return ''
    if x == 0:
        return '0'
    order = int(math.floor(math.log10(abs(x))))
    decimals = sig - 1 - order
    if decimals < 0:
        decimals = 0
    s = f"{x:.{decimals}f}"
    return s.replace('.', ',')

# Sortierreihenfolge für Modelle
MODEL_ORDER = ['Pi4b', 'Pi3b', 'PiZero2', 'PiBanana', 'PiZero', 'Pi1bPlus', 'Pi1b']

# Pfade
df_path = 'output/csv/nist_all.csv'
#out_dir = 'output/xlsx/picnic_only'
out_dir = 'output/xlsx/nist_scaled'

# Output-Verzeichnis neu anlegen
#if os.path.exists(out_dir):
#    shutil.rmtree(out_dir)
#os.makedirs(out_dir, exist_ok=True)

# CSV einlesen
raw = pd.read_csv(df_path)
raw.columns = raw.columns.str.lower()

# Implementation extrahieren (Suffix _OI oder _neon)
raw['impl'] = raw['algorithmus'].str.extract(r'_(OI|neon)$', expand=False).str.upper().fillna('')
# Algorithmus-Spalte säubern: entferne "_OI" und "_neon"
raw['algorithmus'] = raw['algorithmus'].str.replace(r'(_OI|_neon)$', '', regex=True)

# Nur Picnic-Verfahren weiter verarbeiten
picnic_df = raw[raw['algorithmus'].str.lower().str.startswith('picnic')]

# Pro Algorithmus verarbeiten
for algo, grp in picnic_df.groupby('algorithmus'):
    safe = algo.replace('/', '_').replace(' ', '_')
    wb = Workbook()
    ws = wb.active
    ws.title = 'NIST All'

    # Merge für Model und Impl Kopf über 2 Zeilen
    ws.merge_cells(start_row=1, start_column=1, end_row=2, end_column=1)
    cell_model = ws.cell(row=1, column=1, value='Model')
    cell_model.font = FONT_BOLD
    cell_model.alignment = Alignment(horizontal='center', vertical='center')
    for r in (1, 2):
        ws.cell(row=r, column=1).border = border

    ws.merge_cells(start_row=1, start_column=2, end_row=2, end_column=2)
    cell_impl = ws.cell(row=1, column=2, value='Impl')
    cell_impl.font = FONT_BOLD
    cell_impl.alignment = Alignment(horizontal='center', vertical='center')
    for r in (1, 2):
        ws.cell(row=r, column=2).border = border

    # Typ (kem/sig)
    typ = 'kem'
    if 'typ' in grp.columns and grp['typ'].notna().any():
        typ = str(grp['typ'].dropna().iloc[0]).lower()
    sub_map = {'Enc': 'Sign', 'Dec': 'Verify'} if typ == 'sig' else {'Enc': 'Enc', 'Dec': 'Dec'}

    # Roh-Daten inkl. Impl
    df = pd.DataFrame({
        'model':        grp['model'],
        'impl':         grp['impl'],
        'keygen_time':  grp['keygen_time'],
        'enc_time':     grp['enc_time'],
        'dec_time':     grp['dec_time'],
        'keygen_heap':  grp['keygen_heap'],
        'enc_heap':     grp['enc_heap'],
        'dec_heap':     grp['dec_heap'],
        'keygen_stack': grp['keygen_stack'],
        'enc_stack':    grp['enc_stack'],
        'dec_stack':    grp['dec_stack'],
    })
    df['model'] = pd.Categorical(df['model'], categories=MODEL_ORDER, ordered=True)
    df = df.sort_values('model')

    # Dynamische Skalierung basierend auf kleinstem Wert >= Einheit
    times = df[['keygen_time','enc_time','dec_time']].values.flatten()
    nz_t = times[times > 0]
    min_t = nz_t.min() if len(nz_t) else 0
    if min_t >= 1e6:
        t_scale, t_unit = 1e6, 's'
    elif min_t >= 1e3:
        t_scale, t_unit = 1e3, 'ms'
    else:
        t_scale, t_unit = 1, 'µs'
    heaps = df[['keygen_heap','enc_heap','dec_heap']].values.flatten()
    nz_h = heaps[heaps > 0]
    min_h = nz_h.min() if len(nz_h) else 0
    if min_h >= 2**20:
        h_scale, h_unit = 2**20, 'MiB'
    elif min_h >= 2**10:
        h_scale, h_unit = 2**10, 'KiB'
    else:
        h_scale, h_unit = 1, 'B'
    stacks = df[['keygen_stack','enc_stack','dec_stack']].values.flatten()
    nz_s = stacks[stacks > 0]
    min_s = nz_s.min() if len(nz_s) else 0
    if min_s >= 2**20:
        s_scale, s_unit = 2**20, 'MiB'
    elif min_s >= 2**10:
        s_scale, s_unit = 2**10, 'KiB'
    else:
        s_scale, s_unit = 1, 'B'

    # Skalierung und Formatierung
    if t_scale == 1:
        for col in ['keygen_time','enc_time','dec_time']:
            df[col] = df[col].apply(lambda v: '' if pd.isna(v) else str(int(v)))
    else:
        for col in ['keygen_time','enc_time','dec_time']:
            df[col] = df[col].apply(lambda v: format_sig(v / t_scale))
    for col, scale in [('keygen_heap', h_scale), ('enc_heap', h_scale), ('dec_heap', h_scale)]:
        df[col] = df[col].apply(lambda v: format_sig(v / scale))
    for col, scale in [('keygen_stack', s_scale), ('enc_stack', s_scale), ('dec_stack', s_scale)]:
        df[col] = df[col].apply(lambda v: format_sig(v / scale))

    # Header erstellen
    titles = [f'Time ({t_unit})', f'Heap ({h_unit})', f'Stack ({s_unit})']
    headers = ['Model', 'Impl'] + [lbl for t in titles for lbl in ['Key', sub_map['Enc'], sub_map['Dec']]]
    df.columns = headers

    # Gruppen-Header für Time/Heap/Stack
    for (title, start, end) in zip(titles, [3,6,9], [5,8,11]):
        ws.merge_cells(start_row=1, start_column=start, end_row=1, end_column=end)
        for col in range(start, end+1):
            cell = ws.cell(row=1, column=col)
            if col == start:
                cell.value = title
                cell.font = FONT_BOLD
                cell.alignment = Alignment(horizontal='center', vertical='center')
            cell.border = border

    # Untertitel (nur ab Spalte 3)
    for idx, hdr in enumerate(headers, start=1):
        if idx <= 2:
            continue
        c = ws.cell(row=2, column=idx, value=hdr)
        c.font = FONT_BOLD
        c.alignment = Alignment(horizontal='center', vertical='center')
        c.border = border

    # Datenzeilen
    for r, row in enumerate(df.itertuples(index=False), start=3):
        for c_idx, val in enumerate(row, start=1):
            cell = ws.cell(row=r, column=c_idx, value=val)
            cell.font = FONT
            align = Alignment(horizontal='center', vertical='center') if c_idx == 2 else Alignment(horizontal='right', vertical='center')
            cell.alignment = align
            cell.border = border

    # Spaltenbreiten anpassen (Impl extra breit)
    for ci in range(1, len(headers)+1):
        letter = get_column_letter(ci)
        max_len = 0
        for rr in range(2, ws.max_row+1):
            v = ws.cell(row=rr, column=ci).value
            if v:
                max_len = max(max_len, len(str(v)))
        # Impl-Spalte eine Einheit breiter
        extra = 1 if ci == 2 else 0
        ws.column_dimensions[letter].width = max_len + 1 + extra

    # Zeilenhöhe setzen
    for rr in range(1, ws.max_row+1):
        ws.row_dimensions[rr].height = 12

    # Speichern
    wb.save(os.path.join(out_dir, f'{safe}.xlsx'))
    print(f"Erzeugt: {safe}.xlsx")

Erzeugt: Picnic_3l1.xlsx
Erzeugt: Picnic_3l3.xlsx
Erzeugt: Picnic_3l5.xlsx
Erzeugt: Picnic_l1fs.xlsx
Erzeugt: Picnic_l1full.xlsx
Erzeugt: Picnic_l1ur.xlsx
Erzeugt: Picnic_l3fs.xlsx
Erzeugt: Picnic_l3full.xlsx
Erzeugt: Picnic_l3ur.xlsx
Erzeugt: Picnic_l5fs.xlsx
Erzeugt: Picnic_l5full.xlsx
Erzeugt: Picnic_l5ur.xlsx
